# This is the first script that needs to be run on Google Colab

# Before you get started:
1. Click on "Runtime" then "Change runtime Type"
2. Change hardware accelerator to "GPU"

# Install necessary packages

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs

## This will download the data and unzip it to the server
## Do not worry, this will not affect your space on your google drive

In [ ]:
import zipfile, os
def unzip(path_to_zip, out_path):
  with zipfile.ZipFile(path_to_zip, 'r') as zip_ref:
      zip_ref.extractall(out_path)
  return None

### Clone the module from my github and unzip the data

In [ ]:
!git-lfs clone --recurse-submodules -j8 https://github.com/brianmanderson/WIMP_Workshop_2020
os.chdir(os.path.join('.','WIMP_Workshop_2020'))

path_to_data = os.path.join('.','Data.zip')
out_path = os.path.join('.')
print('Unzipping!')
unzip(path_to_data,out_path)
print('Finished!')

### Install a few packages that we will need

In [ ]:
!pip install 'tensorflow-gpu==1.15.0'
!pip install pydicom==1.2.2
!pip install SimpleITK

### Finished!

# DeepBox

## First we need to import a few things, this includes our generator and visualizing module

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os
from Deep_Learning.Base_Deeplearning_Code.Visualizing_Model.Visualing_Model import visualization_model_class
%matplotlib inline

## What do we need? We need a way to generate larges amounts of training data for our model..

In [ ]:
from Deep_Learning.Shape_Maker import Data_Generator, make_rectangle, make_circle
image_size = 64

## The make_rectangle and make_circle will both return circles and rectangles, and the Data_Generator will randomly create circles or rectangles

In [ ]:
plt.imshow(make_rectangle(image_size))

In [ ]:
plt.imshow(make_circle(image_size))

### Our generator essentially continiously creates examples

In [ ]:
train_generator = Data_Generator(image_size=image_size,batch_size=32, num_examples_per_epoch=150)

In [ ]:
x,y = train_generator.__getitem__(0)
print(x.shape)
print(y.shape)

## Now lets make our network!

In [ ]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Activation
from tensorflow.python.keras.optimizers import Adam
import tensorflow.compat.v1 as tf
import tensorflow.python.keras.backend as K
from tensorflow.compat.v1 import Graph, Session, ConfigProto, GPUOptions

### This will make sure multiple networks don't clog up the GPU

In [ ]:
def prep_network():
    K.clear_session()
    gpu_options = tf.GPUOptions(allow_growth=True)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    K.set_session(sess)
    return None

### Representation of network

![DeepBox_Network.png](./Deep_Learning/DeepBox_Network.png)

### Building the network

In [ ]:
prep_network()
num_kernels = 4
kernel_size = (3,3)
model = Sequential([
    Conv2D(num_kernels, kernel_size, 
           input_shape=(image_size, image_size, 1), 
           padding='same',name='Conv_0',activation='sigmoid'),
    MaxPool2D((image_size)), # Pool into a 1x1x4 image
    Flatten(),
    Dense(2,activation='softmax')
])

### Defining loss
#### We are specifying that we care about the categorical cross entropy, with a learning rate of 0.1 (very high)

In [ ]:
model.compile(Adam(lr=1e-1), loss='categorical_crossentropy', metrics=['accuracy'])

### Train
#### We give the model our generator, and tell it to run for 5 epochs

In [ ]:
model.fit_generator(train_generator,epochs=5)

### Evaluate
### We will randomly create 500 examples of rectangles and circles and see how well our model does

In [ ]:
def determine_accuracy(model, image_size= 64, num_examples=1000):
    truth = np.zeros((num_examples,1))
    guess = np.zeros((num_examples,1))
    index = 0
    for _ in range(num_examples//2):
        pred = model.predict(make_rectangle(image_size)[None,...,None])
        guess[index] = np.argmax(pred)
        truth[index] = 1
        index += 1
    for _ in range(num_examples//2):
        pred = model.predict(make_circle(image_size)[None,...,None])
        guess[index] = np.argmax(pred)
        index += 1
    print('Accuracy is {} for {} examples'.format(str((guess==truth).sum()/num_examples),num_examples))

In [ ]:
determine_accuracy(model)

### Lets see how confident it is in it's predictions, generate a random circle or rectangle and see what the confidence is

In [ ]:
rectangle = make_rectangle(image_size)[None,...,None]
circle = make_circle(image_size)[None,...,None]
print('{}% confident'.format(model.predict(rectangle)[...,1][0]*100))
print('{}% confident'.format(model.predict(circle)[...,0][0]*100))

## Lets see what the kernels and activations look like

In [ ]:
Visualizing_Class = visualization_model_class(model=model)

### Say that we only want to look at Conv_0

In [ ]:
Visualizing_Class.define_desired_layers(desired_layer_names=['Conv_0'])

## Kernels

In [ ]:
Visualizing_Class.plot_kernels()

## Activations
### In order to make an activation map we need to provide it with something to predict on

In [ ]:
Visualizing_Class.predict_on_tensor(make_rectangle(image_size)[None,...,None])

In [ ]:
Visualizing_Class.plot_activations()

## How big is this model? Super tiny!!

In [ ]:
model.summary()

# Data curation

### Import some necessary functions

In [ ]:
import os
def return_sys_path():
    path = '.'
    for _ in range(5):
        if 'Pre_Processing' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path
def return_data_path():
    path = '.'
    for _ in range(5):
        if 'Data' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path

In [ ]:
import pydicom, sys
sys.path.append(return_sys_path())
import numpy as np
import SimpleITK as sitk
from Pre_Processing.Distribute_Patients import Separate_files
from Pre_Processing.Dicom_RT_and_Images_to_Mask.Image_Array_And_Mask_From_Dicom_RT import Dicom_to_Imagestack
from Pre_Processing.Dicom_RT_and_Images_to_Mask.Plot_And_Scroll_Images.Plot_Scroll_Images import plot_Image_Scroll_Bar_Image
from Pre_Processing.Make_Single_Images.Make_Single_Images_Class import run_main

## Finding the Data

### Find where we put our data

In [ ]:
data_path = os.path.join(return_data_path(),'Data','Whole_Patients')
print('We have ' + str(len(os.listdir(data_path))) + ' patients!')

## Ensuring contour fidelity...

### Note that we've set 'get_images_mask' to False, this means we won't be getting any of the image data, just looking at the dicom RT files

In [ ]:
Dicom_Reader = Dicom_to_Imagestack(get_images_mask=False)

In [ ]:
Dicom_Reader.down_folder(data_path)

### What ROI names do we have?

#### This will tell us all the unique roi names, hence all_rois

In [ ]:
for roi in Dicom_Reader.all_rois:
    print(roi)

## Make contour associations

#### We have quite a few contour names here.. now, we can either change the ROI names in the RT files, or make an associations file

#### The associations file associates a contour name with another one {'Current contour':'Desired name'}

In [ ]:
associations = {'Liver_BMA_Program_4':'Liver',
                'bma_liver':'Liver',
                'best_liver':'Liver',
                'tried_liver':'Liver'}

### Tell the Dicom_Reader that we want to set the associations, get the images and mask for contour 'Liver'

In [ ]:
Dicom_Reader.set_associations(associations)
Dicom_Reader.set_get_images_and_mask(True)
Dicom_Reader.set_contour_names(['liver'])

### Re-write RTs
#### This is commented out, because if I run it, then the example above won't show any different contour names

In [ ]:
# Dicom_Reader.associations = associations
# for RT in Dicom_Reader.all_RTs:
#     Dicom_Reader.rewrite_RT(RT)

## Pulling images and mask

### We'll first do this with one patient

In [ ]:
patient_data_path = os.path.join(data_path,'ABD_LYMPH_036')
Dicom_Reader.Make_Contour_From_directory(patient_data_path)
print('Done!')

## View images

In [ ]:
%matplotlib inline

### The images and mask are saved within the Dicom_Reader class, so we just have to load them

In [ ]:
Images = Dicom_Reader.ArrayDicom
mask = Dicom_Reader.mask # This is the mask

#### Threshold

In [ ]:
Images[Images<-200] = -200
Images[Images>200] = 200

In [ ]:
plot_Image_Scroll_Bar_Image(Images)

In [ ]:
Images[mask==1] += 300

## Recap

### Checking ROI contour names and making associations

### Loading in image and mask from desired contour name

### Viewing images and mask

## Separate into Train/Test/Validation

### This is also important, but I would recommend using the 'Parallel' approach available in https://github.com/brianmanderson/Dicom_Data_to_Numpy_Arrays
### For ease, this has already been done for you

In [ ]:
def write_data(data_path, out_path, Dicom_Reader,desc = 'TCIA_Liver_Patients'):
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    print(out_path)
    Dicom_Reader.set_description(desc)
    iteration = 0
    for patient in os.listdir(data_path):
        print(patient)
        patient_data_path = os.path.join(data_path,patient)
        out_file = os.path.join(patient_data_path, desc + '_Iteration_' + str(iteration) + '.txt')
        if not os.path.exists(out_file):
            Dicom_Reader.Make_Contour_From_directory(patient_data_path)
            Dicom_Reader.set_iteration(iteration)
            Dicom_Reader.write_images_annotations(out_path)
        iteration += 1
    return None

In [ ]:
output_path = data_path.replace('Whole_Patients','Niftii_Arrays')
#write_data(data_path,output_path, Dicom_Reader)
#Separate_files(output_path) # Separate into a Training/Validation/Test set
#run_main(output_path,extension=5)

# Liver Model

## Import some things

In [ ]:
import os
def return_sys_path():
    path = '.'
    for _ in range(5):
        if 'Deep_Learning' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path
def return_data_path():
    path = '.'
    for _ in range(5):
        if 'Data' in os.listdir(path):
            break
        else:
            path = os.path.join(path,'..')
    return path

In [ ]:
import os, sys
sys.path.append(return_sys_path())
from Deep_Learning.Base_Deeplearning_Code.Data_Generators.Generators import Data_Generator_Class, os
from Deep_Learning.Base_Deeplearning_Code.Keras_Utils.Keras_Utilities import np, dice_coef_3D
from Deep_Learning.Base_Deeplearning_Code.Data_Generators.Image_Processors import *
from Deep_Learning.Base_Deeplearning_Code.Callbacks.Visualizing_Model_Utils import TensorBoardImage
from Deep_Learning.Base_Deeplearning_Code.Callbacks.BMA_Callbacks import Add_LR_To_Tensorboard
from tensorflow.python.keras.callbacks import ModelCheckpoint
# from Utils import ModelCheckpoint, model_path_maker
from Deep_Learning.Base_Deeplearning_Code.Plot_And_Scroll_Images.Plot_Scroll_Images import plot_Image_Scroll_Bar_Image
import tensorflow.compat.v1 as tf
import tensorflow.python.keras.backend as K

In [ ]:
%matplotlib inline

In [ ]:
%load_ext tensorboard

In [ ]:
base = return_sys_path()
data_path = os.path.join(return_data_path(),'Data','Niftii_Arrays')
train_path = [os.path.join(data_path,'Train')]
validation_path = os.path.join(data_path,'Validation')
test_path = os.path.join(data_path,'Test')
model_path = os.path.join(base,'Models')
if not os.path.exists(model_path):
    os.makedirs(model_path)

## We now need some image processors...

#### We will ensure that the images are 256 x 256 (downsampled for speed), normalize them with a mean of 78 and std of 29, add random noise, threshold, and turn into 2 classes

In [ ]:
image_size = 128
image_processors_train = [Ensure_Image_Proportions(image_size,image_size),Repeat_Channel(repeats=3),
                          Normalize_Images(mean_val=78,std_val=29),
                          Threshold_Images(lower_bound=-3.55,upper_bound=3.55, post_load=False, final_scale_value=1),
                          Annotations_To_Categorical(2)]
#Add_Noise_To_Images(variation=np.round(np.arange(start=0, stop=0.3, step=0.1),2)),
image_processors_test = [Ensure_Image_Proportions(image_size,image_size),Normalize_Images(mean_val=78,std_val=29),
                         Repeat_Channel(repeats=3), 
                         Threshold_Images(lower_bound=-3.55,upper_bound=3.55, post_load=False,final_scale_value=1),
                         Annotations_To_Categorical(2)]

In [ ]:
batch_size = 5
train_generator = Data_Generator_Class(by_patient=False,shuffle=True,data_paths=train_path,batch_size=batch_size,
                                       image_processors=image_processors_train)
validation_generator = Data_Generator_Class(by_patient=True,shuffle=True,data_paths=validation_path,batch_size=30,
                                            image_processors=image_processors_test, by_patient_2D=True)

### Lets visualize one of the examples! With batch_size of 5 and shuffle on, it will be 5 random 2D slices

In [ ]:
x,y = train_generator.__getitem__(0)

In [ ]:
plot_Image_Scroll_Bar_Image(x)

### Alright, lets make our model!

In [ ]:
from Deep_Learning.Easy_VGG16_UNet.Keras_Fine_Tune_VGG_16_Liver import VGG_16
from Deep_Learning.Base_Deeplearning_Code.Visualizing_Model.Visualing_Model import visualization_model_class
from tensorflow.python.keras.optimizers import Adam

### This is just a click and play, it builds the VGG16 architecture for you with pre-trained weights

![VGG16_Unet.png](./Deep_Learning/Easy_VGG16_UNet/VGG16_UNet.png)

In [ ]:
K.clear_session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
K.set_session(sess)
network = {'Layer_0': {'Encoding': [64, 64], 'Decoding': [64]},
           'Layer_1': {'Encoding': [128, 128], 'Decoding': [64]},
           'Layer_2': {'Encoding': [256, 256, 256], 'Decoding': [256]},
           'Layer_3': {'Encoding': [512, 512, 512], 'Decoding': [256]},
           'Layer_4': {'Encoding': [512, 512, 512]}}
VGG_model = VGG_16(network=network, activation='relu',filter_size=(3,3))
VGG_model.make_model()
VGG_model.load_weights()
new_model = VGG_model.created_model
model_path = os.path.join(return_sys_path(),'Models')

## These are some tools for visualizing the model

In [ ]:
Visualizing_Class = visualization_model_class(model=new_model, save_images=True, verbose=True)

### Lets look at the activations of block1_conv1, the activation, and output

In [ ]:
Visualizing_Class.define_desired_layers(['block1_conv1','block1_conv1_activation','Output'])

In [ ]:
Visualizing_Class.predict_on_tensor(x[0,...][None,...])

In [ ]:
Visualizing_Class.plot_activations()

In [ ]:
new_model.compile(Adam(lr=5e-5),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

## Freezing pre-trained layers

In [ ]:
def freeze_until_name(model,name):
    set_trainable = False
    for layer in model.layers:
        if layer.name == name:
            set_trainable = True
        layer.trainable = set_trainable
    return model
new_model = freeze_until_name(new_model,'Upsampling0_UNet')

## Checkpoint and run

A checkpoint is a way of assessing the model and determining if we should save it

In [ ]:
model_name = 'VGG_16_Model'
model_path_out = os.path.join(model_path,'VGG_16_frozen')
if not os.path.exists(model_path_out):
    os.makedirs(model_path_out)

In [ ]:
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'best-model.hdf5'), monitor='val_dice_coef_3D', verbose=1, save_best_only=True,
                              save_weights_only=False, mode='max')
# TensorboardImage lets us view the predictions of our model
tensorboard = TensorBoardImage(log_dir=model_path_out, num_images=3,update_freq='epoch', 
                               data_generator=validation_generator, image_frequency=1)
callbacks = [checkpoint, tensorboard]

### Lets view the model real quick

In [ ]:
from tensorflow.python.keras.callbacks_v1 import TensorBoard

In [ ]:
k = TensorBoard(model_path_out)
k.set_model(new_model)

In [ ]:
%tensorboard --logdir {"./Models"}

### Lets train!

In [ ]:
new_model.fit_generator(train_generator,epochs=5, workers=20, max_queue_size=50,
                        validation_data=validation_generator,callbacks=callbacks,
                        steps_per_epoch=200)

In [ ]:
x,y = validation_generator.__getitem__(0)

In [ ]:
pred = new_model.predict(x)

In [ ]:
pred[pred<0.5] = 0
pred[pred>0] = 1

In [ ]:
plot_scroll_Image(pred[...,1])

# Now lets make our own architecture

### First, lets import some necessary functions

In [ ]:
from Deep_Learning.Base_Deeplearning_Code.Models.Keras_Models import my_UNet
from Deep_Learning.Base_Deeplearning_Code.Cyclical_Learning_Rate.clr_callback import CyclicLR
from tensorflow.python.keras.callbacks import ModelCheckpoint
from functools import partial
from tensorflow.python.keras.optimizers import Adam

### Define our convolution and strided blocks, strided is used for downsampling

In [ ]:
activation = {'activation': 'relu'}
kernel = (3,3)
pool_size = (2,2)
#{'channels': x, 'kernel': (3, 3), 'strides': (1, 1),'activation':activation}
conv_block = lambda x: {'convolution': {'channels': x, 'kernel': (3, 3),
                                        'activation': None, 'strides': (1, 1)}}
pooling_downsampling = {'pooling': {'pooling_type': 'Max',
                                    'pool_size': (2, 2), 'direction': 'Down'}}
pooling_upsampling = {'pooling': {'pool_size': (2, 2), 'direction': 'Up'}}

### Our architecture will have 2 main parts in each layer, an 'Encoding' and a 'Decoding' side, 'Encoding' goes down, and 'Decoding' goes up

![Encoding and Decoding.png](./Deep_Learning/Encoding_and_Decoding.png)

### We will now create our layer dictionary, this tells our UNet what to look like

### If Pooling is left {} it will perform maxpooling and upsampling with pooling()

In [ ]:
layers_dict = {}
layers_dict['Layer_0'] = {'Encoding': [],
                          'Decoding': [],
                          'Pooling':
                              {'Encoding': [],
                               'Decoding': []
                               }}
layers_dict['Base'] = []
layers_dict['Final_Steps'] = []

In [ ]:
layers_dict['Layer_0'] = {'Encoding': [conv_block(16),activation,conv_block(16),activation],
                          'Decoding': [conv_block(32),activation,conv_block(32),activation],
                          'Pooling':
                              {'Encoding': [pooling_downsampling],
                               'Decoding': [pooling_upsampling]
                               }}
layers_dict['Base'] = [conv_block(32),activation,conv_block(32),activation]
layers_dict['Final_Steps'] = [conv_block(2),{'activation':'softmax'}]

In [ ]:
K.clear_session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
K.set_session(sess)
new_model = my_UNet(kernel=(3, 3), layers_dict=layers_dict, pool_size=(2, 2), input_size=3, image_size=image_size).created_model

### Name your model and define other things! Send a list of strings and it will make a folder path

In [ ]:
model_name = 'My_New_Model'
model_path_out = os.path.join(model_path,model_name)
if not os.path.exists(model_path_out):
    os.makedirs(model_path_out)

### Lets look at our model

In [ ]:
from tensorflow.python.keras.callbacks_v1 import TensorBoard

In [ ]:
k = TensorBoard(model_path_out)
k.set_model(new_model)

In [ ]:
%tensorboard --logdir {"./Models"}

### Set a learning rate and loss metric, also add any metrics you want to track

In [ ]:
min_lr = 5e-6
max_lr = 1e-3
new_model.compile(Adam(lr=min_lr),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

### This is a checkpoint to save the model if it has the highest dice, also to add images

#### We will specify that we want to watch the validation dice, and save the one with the highest value

In [ ]:
monitor = 'val_dice_coef_3D'
mode = 'max'
checkpoint = ModelCheckpoint(os.path.join(model_path_out,'best-model.hdf5'), monitor=monitor, verbose=1, save_best_only=True,
                             save_weights_only=False, save_freq='epoch', mode=mode)

#### Next, our tensorboard output will add prediction images

In [ ]:
# TensorboardImage lets us view the predictions of our model
tensorboard = TensorBoardImage(log_dir=model_path_out, num_images=1,update_freq='epoch',
                               data_generator=validation_generator, image_frequency=1)


#### CyclicLR will allow us to change the learning rate of the model as it runs, and Add_LR_To_Tensorboard will let us view it later

In [ ]:
steps_per_epoch = len(train_generator)//3
step_size_factor = 10

cyclic_lrate = CyclicLR(base_lr=min_lr, max_lr=max_lr, step_size=steps_per_epoch * step_size_factor, mode='triangular2')
add_lr_to_tensorboard = Add_LR_To_Tensorboard()

### Combine all callbacks

In [ ]:
callbacks = [cyclic_lrate, add_lr_to_tensorboard, tensorboard, checkpoint]

In [ ]:
new_model.fit_generator(train_generator,epochs=10, workers=10, max_queue_size=200, validation_data=validation_generator,
                       callbacks=callbacks, steps_per_epoch=steps_per_epoch)

In [ ]:
%tensorboard --logdir {"./Models"}